# Import the libraries

In [1]:
pip install paretoset

In [2]:
import os
import cv2
import math
import random
import copy
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt
from moviepy.editor import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras import backend as K
from sklearn import preprocessing
import time
import pandas as pd
import gc
import multiprocessing
from multiprocessing import Process, Queue
from paretoset import paretoset

  if event.key is 'enter':



# Definition of programmer's parameters (Controller's block)

In [3]:
#@markdown ##Synopses-based Training Optimization Configuration
# Percentage of the dataset that will be used for training
sample_size_low = 0.05 #@param {type:"number"}
sample_size_step = 0.1 #@param {type:"number"}
sample_size_high = 1 #@param {type:"number"}

# The ID of the method that will be employed during sampling
# 0: Simple reservoir sampling in our initial training dataset
# 1: Reservoir sampling in each class based on the number of samples (per class)
sampling_method_id = 1 #@param {type:"integer"}

#@markdown ##NN Architecture Configuration
total_num_of_layers = 5 #@param {type:"integer"}

set_of_layers = ['conv', 'pool', 'lstm', 'gru', 'rnn', 'dense']

# Number of epochs for the training process
# One Epoch is when an ENTIRE (training) dataset is passed forward and backward through the neural network only once.
# NOTE: An epoch is comprised of one or more batches.
num_of_epochs_low = 1 #@param {type:"slider", min:1, max:30, step:1}
num_of_epochs_step = 4 #@param {type:"integer"}
num_of_epochs_high = 20 #@param {type:"slider", min:1, max:31, step:1}

# The value for the learning rate for the training process (it is inserted to the Adam optimizer)
# During the optimization, the algorithm needs to take a series of tiny steps to descend the error mountain in order to minimize the error.
# The direction of the step is determined by the gradient, while the step size is determined by the learning rate.
lr_low = 1e-3 #@param {type:"number"}
lr_high = 1e-3 #@param {type:"number"}
lr_list = np.geomspace(lr_low, lr_high, num = int(np.log10(lr_high) - np.log10(lr_low)) + 1).tolist()

# The size of the batch for the training process
# Total number of training examples present in a single batch.
# or, number of samples processed before the model is updated.
size_of_batch_low = 10 #@param {type:"integer"}
size_of_batch_step = 16 #@param {type:"integer"}
size_of_batch_high = 10 #@param {type:"integer"}

#@markdown ##Tradeoff score Configuration
# Maximum training time above which we cancel the experiment (in seconds)
# It is used in the score (accuracy--speed tradeoff) formula
theta_parameter = 10 #@param {type:"integer"}

# Weight for the accuracy of the model. Max value: 0.99
# There is also the weight of training speed of the model (1 - lamda_acc)
# It is used in the score (accuracy--speed tradeoff) formula
lamda_acc = 0.1 #@param {type:"number"}

#@markdown ##Other Configuration(s)

# Height and width to which each video will be resized
IMAGE_HEIGHT = 64 #@param {type:"integer"}
IMAGE_WIDTH = 64 #@param {type:"integer"}

# Number of frames of a SINGLE video that will be fed to the model (as a single sequence)
SEQUENCE_LENGTH = 20 #@param {type:"integer"}

# Classes that will be used
CLASSES_LIST = ["Punch", "Skijet", "WalkingWithDog", "HorseRace", "Diving", "PushUps", "Kayaking", "PlayingPiano", "Rowing", "Skiing"]

CONV_NEURONS_CONST = 16 #@param {type:"integer"}
CONV_NEURONS_BOUND = 64 #@param {type:"integer"}
UNITS_CONST = 32 #@param {type:"integer"}
UNITS_BOUND = 32 #@param {type:"integer"}
DENSE_NEURONS_CONST = 128 #@param {type:"integer"}
DENSE_NEURONS_BOUND = 32 #@param {type:"integer"}

# Number of samples-architectures that pass generation to generation
g2g_promotions = 6 #@param {type:"integer"}

# Percentage of architectures that pass identical to the next generation (no expansion is performed)
g2g_percentage = 0.1 #@param {type:"number"}

mutation_lr_low = 0.0001
mutation_lr_high = 0.01

mutation_batch_low = 16
mutation_batch_high = 64


# Download & Visualize our Data

It takes a while...

In [4]:
!wget --no-check-certificate https://www.crcv.ucf.edu/data/UCF50.rar
!unrar x UCF50.rar

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
Extracting  UCF50/HorseRace/v_HorseRace_g16_c03.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g16_c04.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g16_c05.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g17_c01.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g17_c02.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g17_c03.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g17_c04.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g17_c05.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g18_c01.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g18_c02.avi                       23%  OK 
Extracting  UCF50/HorseRa

In [5]:
plt.figure(figsize = (20, 20))

# Get the names of all available classes
all_classes_names = os.listdir('UCF50')

# Generate a list of 20 random variables (between 0 - #num_of_classes)
random_range = random.sample(range(len(all_classes_names)), 20)

# Iterate over random variables
for counter, random_index in enumerate(random_range, 1):
  # Retrieve the name of the corresponding class
  selected_class_Name = all_classes_names[random_index]

  # Retrieve the list of all videos that are contained in the selected class and select a single one randomly
  video_files_names_list = os.listdir(f'UCF50/{selected_class_Name}')
  selected_video_file_name = random.choice(video_files_names_list)
  video_reader = cv2.VideoCapture(f'UCF50/{selected_class_Name}/{selected_video_file_name}')

  # Read the first frame from the selected video
  _, bgr_frame = video_reader.read()
  video_reader.release()

  # Convert frame from BGR to RGB
  rgb_frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB)

  # Write the corresponding label
  cv2.putText(rgb_frame, selected_class_Name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

  # Plot the frame
  plt.subplot(5, 4, counter);plt.imshow(rgb_frame);plt.axis('off')

Output hidden; open in https://colab.research.google.com to view.

# Pre-process Part

In [6]:
DATASET_DIR = "UCF50"

Function for extracting the required frames from a video after employing resizing and normalization procedures

In [7]:
def frames_extraction(video_path):
  # Store the video frames
  frames_list = []

  # Read the video
  video_reader = cv2.VideoCapture(video_path)

  # Get total number of frames (of this video)
  video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

  # Calculate the interval after which frames will be stored (the step)
  skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

  # Iterate over video-frames
  for frame_counter in  range(SEQUENCE_LENGTH):
    # Adjust the pointer of current frame
    video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

    # Read the corresponding frame
    success, frame = video_reader.read()

    if not success:
      break

    # Resize-normalize the frame and save it to the corresponding list
    resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
    normalized_frame = resized_frame / 255
    frames_list.append(normalized_frame)

  video_reader.release()

  return frames_list

Function that creates the dataset that will be used

In [8]:
def create_dataset():
  # Lists that contain the extracted features, the labels and the paths of the videos
  features = []
  labels = []
  video_files_paths = []

  # Iterate through all (selected) classes
  for class_index, class_name in enumerate(CLASSES_LIST):
    print(f'Extracting Data of Class: {class_name}')

    # Get the videos that are contained in each (selected) class
    files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
    for file_name in files_list:
      video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
      frames = frames_extraction(video_file_path)
      if len(frames) == SEQUENCE_LENGTH:
        features.append(frames)
        labels.append(class_index)
        video_files_paths.append(video_file_path)

  # Convert lists to numpy arrays
  features = np.asarray(features)
  labels = np.asarray(labels)

  return features, labels, video_files_paths


In [9]:
# Create the dataset
features, labels, video_files_paths = create_dataset()

Extracting Data of Class: Punch
Extracting Data of Class: Skijet
Extracting Data of Class: WalkingWithDog
Extracting Data of Class: HorseRace
Extracting Data of Class: Diving
Extracting Data of Class: PushUps
Extracting Data of Class: Kayaking
Extracting Data of Class: PlayingPiano
Extracting Data of Class: Rowing
Extracting Data of Class: Skiing


# Split the Data into Train and Test sets

In [10]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.25, shuffle = True)

# Sampling Part

In [11]:
# A function that prints the occurence of each class in a list
def print_times_per_label(lst, labels_all):
  # Get unique labels in our training dataset
  unique_labels = np.unique(labels_all)
  for i in range(0, len(unique_labels)):
    print("Class", unique_labels[i], "has", lst.count(i), "samples in our dataset...")

In [12]:
# A function to randomly select k items from stream[0..n-1].
def reservoir_sampling(stream, n, k):
  i = 0     # index for elements in stream[]

  # reservoir[] is the output array.
  # Initialize it with first k elements from stream[]
  reservoir = [0] * k

  for i in range(k):
    reservoir[i] = stream[i]

  # Iterate from the (k+1)th element to Nth element
  while(i < n):
    # Pick a random index from 0 to i.
    j = random.randrange(i+1)

    # If the randomly picked
    # index is smaller than k,
    # then replace the element
    # present at the index
    # with new element from stream
    if(j < k):
      reservoir[j] = stream[i]
    i+=1

  return reservoir

In [13]:
# A function that finds the size of each reservoir for every class depending on its occurence in the initial dataset
# and returns the unique labels that exist in our dataset along with the corresponding percentage
def reservoir_size_per_class(init_labels):

  # Get unique labels and their counts (how many times they appear) in our training dataset
  unique_labels, counts = np.unique(init_labels, return_counts = True)

  # Transform to list
  unique_labels_lst = unique_labels.tolist()
  counts_lst = counts.tolist()

  perc_per_class = []
  for i in range(len(unique_labels_lst)):
    perc_per_class.append(counts_lst[i]/len(init_labels))

  # print(perc_per_class)

  return perc_per_class, unique_labels_lst

In [14]:
def sampling_method(sampling_method_id, features_train, labels_train, perc_train):
  print("Percentage of filtering in our training dataset was set:")
  print(perc_train)
  print("Pre-sampling training data:")
  print_times_per_label(labels_train.tolist(), np.unique(labels_train))
  if sampling_method_id == 0:
    # Simple reservoir sampling over the whole training dataset
    # Total size of the stream (or training dataset)
    n_train = len(features_train)

    # Number of samples that will be drawn
    k_train = int(n_train * perc_train)

    # Use the indexes of dataset in order to decide which samples will be drawn
    idx_tmp_train_list = list(range(0, n_train))

    # Find the indexes in order to construct the dataset that will be used during the training process
    idx_train = reservoir_sampling(idx_tmp_train_list, n_train, k_train)
  else:
    # Reservoir sampling in each class based on the number of samples (per class) that exist in the initial dataset
    # Find the size of each reservoir for every class depending on its occurence in the initial training dataset
    class_perc, unique_ids = reservoir_size_per_class(labels_train)

    # Stores the indexes (from all classes) in order to construct the dataset that will be used during the training process
    idx_train = []

    # Run for every single class the reservoir sampling seperately
    for i in range(0, len(unique_ids)):
      # Find the locations of each sample belonging to our class of interest
      tmp = np.where(labels_train == unique_ids[i])
      idx_of_class = tmp[0].tolist()

      # Run the reservoir sampling for the class of interest
      sampled_idx_of_class = reservoir_sampling(idx_of_class, len(idx_of_class), int(len(features_train) * perc_train * class_perc[i]))

      # Store the (sampled) samples from this class
      for j in range(0, len(sampled_idx_of_class)):
        idx_train.append(sampled_idx_of_class[j])

  # Store the corresponding images and labels from training dataset based on the sampled indexes
  train_images_lst = []
  for i in idx_train:
    train_images_lst.append(features_train[i])

  train_labels_lst = []
  for i in idx_train:
    train_labels_lst.append(labels_train[i])

  # Check the occurence of each class in the final training dataset
  print("====================================================")
  print("Post-sampling dataset:")
  print_times_per_label(train_labels_lst, labels_train)

  # Tranfsorm the lists that we stored our samples into arrays
  x_train = np.asarray(train_images_lst)
  y_train = np.asarray(train_labels_lst)

  # Verify that the desired filtering was performed in both datasets
  print("====================================================")
  print("Training dataset before sampling:")
  print(features_train.shape)
  print(labels_train.shape)
  print("Training dataset after sampling:")
  print(x_train.shape)
  print(y_train.shape)

  return x_train, y_train

# Create our model

In [15]:
def CrowdingDist(df, objective_columns):
    """
    :param df: DataFrame containing samples with features and objectives
    :param objective_columns: List of column names representing the objective values
    :return: List of crowding distances for each sample

    The crowding-distance computation requires sorting the population according to each objective function value
    in ascending order of magnitude. Thereafter, for each objective function, the boundary solutions (solutions with
    smallest and largest function values) are assigned an infinite distance value. All other intermediate solutions
    are assigned a distance value equal to the absolute normalized difference in the function values of two adjacent solutions.
    """
    n_samples = len(df)
    print(n_samples)
    distances = [0.0] * n_samples
    # create keys for fitness values
    crowd = [(tuple(row), i) for i, row in df.iterrows()]

    for objective_column in objective_columns:
        # Get the column position for the specified objective column
        obj_col_position = df.columns.get_loc(objective_column)

        crowd.sort(key=lambda element: element[0][obj_col_position])

        # After sorting, boundary solutions are assigned Inf
        # distances[crowd[0][1]] = float("Inf")
        # distances[crowd[-1][1]] = float("inf")

        if crowd[-1][0][obj_col_position] == crowd[0][0][obj_col_position]:  # If objective values are the same, skip this loop
            continue
        # normalization (max - min) as Denominator
        norm = float(crowd[-1][0][obj_col_position] - crowd[0][0][obj_col_position])
        # calculate each individual's Crowding Distance of i-th objective
        # technique: shift the list and zip
        for prev, cur, next_ in zip(crowd[:-2], crowd[1:-1], crowd[2:]):
            distances[cur[1]] += (next_[0][obj_col_position] - prev[0][obj_col_position]) / norm  # sum up the distance of i-th individual along each of the objectives

    return distances

In [16]:
def recreate_model(layers_lst, layer2add, CONV_NEURONS_CONST, UNITS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, UNITS_BOUND, DENSE_NEURONS_BOUND):

  # Initialize a sequential model
  model = tf.keras.models.Sequential()

  # Define the number of neurons for conv and dense layers and the number of units for lstm-gru-rnn
  conv_tmp2 = CONV_NEURONS_CONST
  units_tmp2 = UNITS_CONST
  dense_tmp2 = DENSE_NEURONS_CONST

  # Find the type of the next and the previous layer because you need different configurations
  for count, layer in enumerate(layers_lst):
    if count == 0 and len(layers_lst) > 1:
      previous_layer_tmp = 'no'
      next_layer_tmp = layers_lst[count + 1]
    elif count == 0:
      previous_layer_tmp = 'no'
      next_layer_tmp = 'no'
    elif count == len(layers_lst) - 1:
      next_layer_tmp = 'no'
      previous_layer_tmp = layers_lst[count - 1]
    else:
      previous_layer_tmp = layers_lst[count - 1]
      next_layer_tmp = layers_lst[count + 1]

    # Recreate the so-far-model
    # First layer conv
    if layer == 'conv' and count == 0:
      model.add(TimeDistributed(Conv2D(int(conv_tmp2), (3, 3), padding = 'same', activation = 'relu'), input_shape = (SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
      conv_tmp2 = conv_tmp2 * 2
    # First layer lstm-gru-rnn (change the shape of the input) and next or 2-be-added layer lstm-gru-rnn (should add the 'return conf')
    elif ((layer == 'lstm' or layer == 'gru' or layer == 'rnn') and (((count == 0) and len(layers_lst) == 1 and (layer2add == 'lstm' or layer2add == 'gru' or layer2add == 'rnn')) or ((count == 0) and (next_layer_tmp == 'lstm' or next_layer_tmp == 'gru' or next_layer_tmp == 'rnn')))):
      if layer == 'lstm':
        model.add(tf.keras.layers.LSTM(int(units_tmp2), return_sequences = True, input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT * IMAGE_WIDTH * 3)))
      elif layer == 'gru':
        model.add(tf.keras.layers.GRU(int(units_tmp2), return_sequences = True, input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT * IMAGE_WIDTH * 3)))
      else:
        model.add(tf.keras.layers.SimpleRNN(int(units_tmp2), return_sequences = True, input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT * IMAGE_WIDTH * 3)))
      units_tmp2 = units_tmp2 / 2
    # First layer lstm-gru-rnn (change the shape of the input)
    elif ((layer == 'lstm' or layer == 'gru' or layer == 'rnn') and count == 0):
      if layer == 'lstm':
        model.add(tf.keras.layers.LSTM(int(units_tmp2), input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT * IMAGE_WIDTH * 3)))
      elif layer == 'gru':
        model.add(tf.keras.layers.GRU(int(units_tmp2), input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT * IMAGE_WIDTH * 3)))
      else:
        model.add(tf.keras.layers.SimpleRNN(int(units_tmp2), input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT * IMAGE_WIDTH * 3)))
      units_tmp2 = units_tmp2 / 2
    # First layer densse (change the shape of the input)
    elif layer == 'dense' and count == 0:
      model.add(tf.keras.layers.Dense(int(dense_tmp2), activation='relu', input_shape=(SEQUENCE_LENGTH * IMAGE_HEIGHT * IMAGE_WIDTH * 3,)))
      dense_tmp2 = dense_tmp2 / 2
    # For the remaining layers
    else:
      if layer == 'conv':
        # Add a conv layer by doubling its neurons if they do not violate our user-defined bound
        if conv_tmp2 <= CONV_NEURONS_BOUND:
          model.add(TimeDistributed(Conv2D(int(conv_tmp2), (3, 3), padding = 'same', activation='relu')))
          conv_tmp2 = conv_tmp2 * 2
        else:
          model.add(TimeDistributed(Conv2D(int(CONV_NEURONS_BOUND), (3, 3), padding = 'same', activation='relu')))
          conv_tmp2 = CONV_NEURONS_BOUND
      elif layer == 'pool':
        # Add a pool layer
        model.add(TimeDistributed(MaxPooling2D((4, 4))))
      elif layer == 'lstm':
        # If the previous layer is conv or pool add a flatten layer first
        if previous_layer_tmp == 'conv' or previous_layer_tmp == 'pool':
          model.add(TimeDistributed(BatchNormalization()))
          model.add(TimeDistributed(Flatten()))
        # Add a lstm layer by reducing (* 0.5) its units if they do not violate our user-defined bound
        if units_tmp2 >= UNITS_BOUND:
          # If the next layer is dense then do not return sequences
          if next_layer_tmp == 'dense' or (layer2add == 'dense' and count == len(layers_lst) - 1):
            model.add(tf.keras.layers.LSTM(int(units_tmp2)))
          else:
            model.add(tf.keras.layers.LSTM(int(units_tmp2), return_sequences = True))
          units_tmp2 = units_tmp2 / 2
        else:
          # If the next layer is dense then do not return sequences
          if next_layer_tmp == 'dense' or (layer2add == 'dense' and count == len(layers_lst) - 1):
            model.add(tf.keras.layers.LSTM(int(UNITS_BOUND)))
          else:
            model.add(tf.keras.layers.LSTM(int(UNITS_BOUND), return_sequences = True))
          units_tmp2 = UNITS_BOUND
      elif layer == 'gru':
        # If the previous layer is conv or pool add a flatten layer first
        if previous_layer_tmp == 'conv' or previous_layer_tmp == 'pool':
          model.add(TimeDistributed(BatchNormalization()))
          model.add(TimeDistributed(Flatten()))
        # Add a gru layer by reducing (* 0.5) its units if they do not violate our user-defined bound
        if units_tmp2 >= UNITS_BOUND:
          # If the next layer is dense then do not return sequences
          if next_layer_tmp == 'dense' or (layer2add == 'dense' and count == len(layers_lst) - 1):
            model.add(tf.keras.layers.GRU(int(units_tmp2)))
          else:
            model.add(tf.keras.layers.GRU(int(units_tmp2), return_sequences = True))
          units_tmp2 = units_tmp2 / 2
        else:
          # If the next layer is dense then do not return sequences
          if next_layer_tmp == 'dense' or (layer2add == 'dense' and count == len(layers_lst) - 1):
            model.add(tf.keras.layers.GRU(int(UNITS_BOUND)))
          else:
            model.add(tf.keras.layers.GRU(int(UNITS_BOUND), return_sequences = True))
          units_tmp2 = UNITS_BOUND
      elif layer == 'rnn':
        # If the previous layer is conv or pool add a flatten layer first
        if previous_layer_tmp == 'conv' or previous_layer_tmp == 'pool':
          model.add(TimeDistributed(BatchNormalization()))
          model.add(TimeDistributed(Flatten()))
        # Add a rnn layer by reducing (* 0.5) its units if they do not violate our user-defined bound
        if units_tmp2 >= UNITS_BOUND:
          # If the next layer is dense then do not return sequences
          if next_layer_tmp == 'dense' or (layer2add == 'dense' and count == len(layers_lst) - 1):
            model.add(tf.keras.layers.SimpleRNN(int(units_tmp2)))
          else:
            model.add(tf.keras.layers.SimpleRNN(int(units_tmp2), return_sequences = True))
          units_tmp2 = units_tmp2 / 2
        else:
          # If the next layer is dense then do not return sequences
          if next_layer_tmp == 'dense' or (layer2add == 'dense' and count == len(layers_lst) - 1):
            model.add(tf.keras.layers.SimpleRNN(int(UNITS_BOUND)))
          else:
            model.add(tf.keras.layers.SimpleRNN(int(UNITS_BOUND), return_sequences = True))
          units_tmp2 = UNITS_BOUND
      else:
        if previous_layer_tmp == 'conv' or previous_layer_tmp == 'pool':
          model.add(Flatten())
        # Add a dense layer by reducing (* 0.5) its neurons if they do not violate our user-defined bound
        if dense_tmp2 >= DENSE_NEURONS_BOUND:
          model.add(tf.keras.layers.Dense(int(dense_tmp2), activation='relu'))
          dense_tmp2 = dense_tmp2 / 2
        else:
          model.add(tf.keras.layers.Dense(int(DENSE_NEURONS_BOUND), activation='relu'))
          dense_tmp2 = DENSE_NEURONS_BOUND

  return model, conv_tmp2, units_tmp2, dense_tmp2

In [17]:
def my_evaluate_first_phase(q, layer, x_train, y_train, features_test, labels_test, epochs, lr, size_of_batch):
  # Initialize a sequential network
  model = tf.keras.models.Sequential()

  # Define the number of neurons for conv and dense layers and the number of units for lstm-gru-rnn
  conv_tmp = CONV_NEURONS_CONST
  units_tmp = UNITS_CONST
  dense_tmp = DENSE_NEURONS_CONST

  # First layer since this method is called only on the first iteration, i.e., the one-layered-networks
  # If the first layer is conv then reshape the input and add a flatten layer
  if layer == 'conv':
    model.add(TimeDistributed(Conv2D(int(conv_tmp), (3, 3), padding = 'same', activation = 'relu'), input_shape = (SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    conv_tmp = conv_tmp * 2
    model.add(Flatten())
    reshaped_x_train = x_train
    reshaped_x_test = features_test
  # If the first layer is lstm-gru-rnn then reshape the input
  elif layer == 'lstm' or layer == 'gru' or layer == 'rnn':
    num_samples, num_frames, height, width, channels = x_train.shape
    if layer == 'lstm':
      model.add(tf.keras.layers.LSTM(int(units_tmp), input_shape=(num_frames, height * width * channels)))
    elif layer == 'gru':
      model.add(tf.keras.layers.GRU(int(units_tmp), input_shape=(num_frames, height * width * channels)))
    else:
      model.add(tf.keras.layers.SimpleRNN(int(units_tmp), input_shape=(num_frames, height * width * channels)))
    units_tmp = units_tmp / 2
    reshaped_x_train = x_train.reshape(num_samples, num_frames, height * width * channels)
    num_samples, num_frames, height, width, channels = features_test.shape
    reshaped_x_test = features_test.reshape(num_samples, num_frames, height * width * channels)
  # If the first layer is dense then reshape the input
  #elif layer == 'dense':
  else:
    num_samples, num_frames, height, width, channels = x_train.shape
    model.add(tf.keras.layers.Dense(int(dense_tmp), activation='relu', input_shape=(num_frames * height * width * channels,)))
    dense_tmp = dense_tmp / 2
    reshaped_x_train = x_train.reshape(num_samples, num_frames * height * width * channels)
    num_samples, num_frames, height, width, channels = features_test.shape
    reshaped_x_test = features_test.reshape(num_samples, num_frames * height * width * channels)

  # Add manually a dense layer in order to evaluate the network
  model.add(tf.keras.layers.Dense(len(CLASSES_LIST), activation='softmax'))

  model.compile(optimizer=Adam(lr), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
  model.summary()

  start = time.time()

  blackbox = model.fit(x = reshaped_x_train,
                      y = y_train,
                      epochs = epochs,
                      batch_size = size_of_batch
                      )
  stop = time.time()

  # Compute the training speed of this CNN architecture
  tr_time = stop - start

  # Compute the accuracy of our training model in the testing dataset
  test_loss, test_acc = model.evaluate(reshaped_x_test,  labels_test, verbose=2)

  # Compute the metric that captures the accuracy--speed tradeoff
  tradeOff_metric = lamda_acc * test_acc - (1 - lamda_acc) * math.tanh(tr_time/theta_parameter - 1)

  # Linear formula
  # tradeOff_metric = lamda_acc * test_acc + (1 - lamda_acc) * (1 - min(1, tr_time/theta_parameter))

  # Multiplicative formula
  # tradeOff_metric = test_acc ** lamda_acc * (1 - min(1, tr_time/theta_parameter)) ** (1 - lamda_acc)

  # Harmonic mean
  # tradeOff_metric = (2 * test_acc * (1 - min(1, tr_time/theta_parameter))) / (test_acc + 1 - min(1, tr_time/theta_parameter) + 1e-5)

  # Sigmoid
  # tradeOff_metric = (1 / (1 + np.exp(-test_acc))) * (1 - 1 / (1 + np.exp(-min(1, tr_time/theta_parameter))))

  # Piecewise
  # tradeOff_metric = test_acc if tr_time <= theta_parameter else test_acc * max(0, 1 - 0.5 * (tr_time - theta_parameter))

  # F1 like
  # alpha = 2
  # tradeOff_metric = ((1 + alpha**2) * test_acc * min(1, tr_time/theta_parameter)) / (alpha ** 2 * test_acc + min(1, tr_time/theta_parameter))

  # Exp decay penalty
  # tradeOff_metric = test_acc * np.exp(-0.5 * min(1, tr_time/theta_parameter))

  # inverse additive penalty
  # tradeOff_metric = test_acc / (1 + 0.5 * min(1, tr_time/theta_parameter))

  # min-based tradeoff
  # tradeOff_metric = min(test_acc, min(1, tr_time/theta_parameter))

  # product with complement
  # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter))

  # square root compensation
  # tradeOff_metric = np.sqrt(test_acc) * np.sqrt(1 - min(1, tr_time/theta_parameter))

  # jacard style normalization
  # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter)) / (test_acc + (1 - min(1, tr_time/theta_parameter)) - test_acc * (1 - min(1, tr_time/theta_parameter)))

  # log-sigmoid dominance
  # tradeOff_metric = 1 / (1 + np.exp(-0.5 * (test_acc - (1 - min(1, tr_time/theta_parameter)))))

  del model

  print("========================== EDW EINAI TO MULTI1111111111111111.......")
  print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
  print(f"Training time: ", tr_time)
  print(tradeOff_metric)

  q.put([test_acc, tr_time, tradeOff_metric])

In [18]:
def my_evaluate_rest_phase(q, x_train, y_train, features_test, labels_test, layer, current_df, CONV_NEURONS_CONST, UNITS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, UNITS_BOUND, DENSE_NEURONS_BOUND):
  error_flag = -1

  # Recreate the network that consist of the best layers that we found in each of the previous steps/iterations
  model, conv_tmp, units_tmp, dense_tmp = recreate_model(current_df['LayerType'], layer, CONV_NEURONS_CONST, UNITS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, UNITS_BOUND, DENSE_NEURONS_BOUND)

  try:
    # If the to-be-added-layer is conv
    if layer == 'conv':
      # Add a conv layer by doubling its neurons if they do not violate our user-defined bound
      if conv_tmp <= CONV_NEURONS_BOUND:
        model.add(TimeDistributed(Conv2D(int(conv_tmp), (3, 3), padding = 'same', activation = 'relu')))
        conv_tmp = conv_tmp * 2
      else:
        model.add(TimeDistributed(Conv2D(int(CONV_NEURONS_BOUND), (3, 3), padding = 'same', activation='relu')))
    # If the to-be-added-layer is pool
    elif layer == 'pool':
      model.add(TimeDistributed(MaxPooling2D((4, 4))))
    # If the to-be-added-layer is lstm
    elif layer == 'lstm':
      # If the previous layer is either conv or pool then add first a flatten layer
      if current_df['LayerType'][-1] == 'conv' or current_df['LayerType'][-1] == 'pool':
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Flatten()))
      # Add a lstm layer by reducing (* 0.5) its units if they do not violate our user-defined bound
      if units_tmp >= UNITS_BOUND:
        model.add(tf.keras.layers.LSTM(int(units_tmp)))
        units_tmp = units_tmp / 2
      else:
        model.add(tf.keras.layers.LSTM(int(UNITS_BOUND)))
        units_tmp = UNITS_BOUND
    # If the to-be-added-layer is gru
    elif layer == 'gru':
      # If the previous layer is either conv or pool then add first a flatten layer
      if current_df['LayerType'][-1] == 'conv' or current_df['LayerType'][-1] == 'pool':
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Flatten()))
      # Add a gru layer by reducing (* 0.5) its units if they do not violate our user-defined bound
      if units_tmp >= UNITS_BOUND:
        model.add(tf.keras.layers.GRU(int(units_tmp)))
        units_tmp = units_tmp / 2
      else:
        model.add(tf.keras.layers.GRU(int(UNITS_BOUND)))
        units_tmp = UNITS_BOUND
    # If the to-be-added-layer is rnn
    elif layer == 'rnn':
      # If the previous layer is either conv or pool then add first a flatten layer
      if current_df['LayerType'][-1] == 'conv' or current_df['LayerType'][-1] == 'pool':
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Flatten()))
      # Add a rnn layer by reducing (* 0.5) its units if they do not violate our user-defined bound
      if units_tmp >= UNITS_BOUND:
        model.add(tf.keras.layers.SimpleRNN(int(units_tmp)))
        units_tmp = units_tmp / 2
      else:
        model.add(tf.keras.layers.SimpleRNN(int(UNITS_BOUND)))
        units_tmp = UNITS_BOUND
    # If the to-be-added-layer is dense
    else:
      # If the next to-be-added-layer is dense and the previous layer is either conv or pool then add manually a flatten layer
      if current_df['LayerType'][-1] == 'conv' or current_df['LayerType'][-1] == 'pool':
        model.add(Flatten())
      # Add a dense layer by reducing (* 0.5) its neurons if they do not violate our user-defined bound
      if dense_tmp >= DENSE_NEURONS_BOUND:
        model.add(tf.keras.layers.Dense(int(dense_tmp), activation='relu'))
        dense_tmp = dense_tmp / 2
      else:
        model.add(tf.keras.layers.Dense(int(DENSE_NEURONS_BOUND), activation='relu'))
        dense_tmp = DENSE_NEURONS_BOUND

    # If the just-added-layer was conv or pool then add manually a flatten layer
    if layer == 'conv' or layer == 'pool':
      model.add(Flatten())

    # Softmax is an activation function that is used mainly for classification tasks
    # It normalizes the input vector into a probability distribution  that is proportional to the exponential of the input numbers.
    model.add(tf.keras.layers.Dense(len(CLASSES_LIST), activation = "softmax"))

  except ValueError:
    print("No valid input...:(")
    error_flag = 1

  if error_flag == -1:
    model.compile(optimizer=Adam(current_df['LearningRate']), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    model.summary()

    # Here we reshape the input of the network based on the type of the first layer of the network
    # If the first layer is conv
    if current_df['LayerType'][0] == 'conv':
      reshaped_x_train = x_train
      reshaped_x_test = features_test
    # If the first layer is lstm-gru-rnn
    elif current_df['LayerType'][0] == 'lstm' or current_df['LayerType'][0] == 'gru' or current_df['LayerType'][0] == 'rnn':
      num_samples, num_frames, height, width, channels = x_train.shape
      reshaped_x_train = x_train.reshape(num_samples, num_frames, height * width * channels)
      num_samples, num_frames, height, width, channels = features_test.shape
      reshaped_x_test = features_test.reshape(num_samples, num_frames, height * width * channels)
    # If the first layer is dense
    else:
      num_samples, num_frames, height, width, channels = x_train.shape
      reshaped_x_train = x_train.reshape(num_samples, num_frames * height * width * channels)
      num_samples, num_frames, height, width, channels = features_test.shape
      reshaped_x_test = features_test.reshape(num_samples, num_frames * height * width * channels)

    start = time.time()

    blackbox = model.fit(x=reshaped_x_train,
                        y=y_train,
                        epochs=current_df['Epochs'],
                        batch_size=current_df['BatchSize']
                        )
    stop = time.time()

    # Compute the training speed of this CNN architecture
    tr_time = stop - start

    # Compute the accuracy of our training model in the testing dataset
    test_loss, test_acc = model.evaluate(reshaped_x_test,  labels_test, verbose=2)

    # Compute the metric that captures the accuracy--speed tradeoff
    tradeOff_metric = lamda_acc * test_acc - (1 - lamda_acc) * math.tanh(tr_time/theta_parameter - 1)

    # Linear formula
    # tradeOff_metric = lamda_acc * test_acc + (1 - lamda_acc) * (1 - min(1, tr_time/theta_parameter))

    # Multiplicative formula
    # tradeOff_metric = test_acc ** lamda_acc * (1 - min(1, tr_time/theta_parameter)) ** (1 - lamda_acc)

    # Harmonic mean
    # tradeOff_metric = (2 * test_acc * (1 - min(1, tr_time/theta_parameter))) / (test_acc + 1 - min(1, tr_time/theta_parameter) + 1e-5)

    # Sigmoid
    # tradeOff_metric = (1 / (1 + np.exp(-test_acc))) * (1 - 1 / (1 + np.exp(-min(1, tr_time/theta_parameter))))

    # Piecewise
    # tradeOff_metric = test_acc if tr_time <= theta_parameter else test_acc * max(0, 1 - 0.5 * (tr_time - theta_parameter))

    # # F1 like
    # alpha = 2
    # tradeOff_metric = ((1 + alpha**2) * test_acc * min(1, tr_time/theta_parameter)) / (alpha ** 2 * test_acc + min(1, tr_time/theta_parameter))

    # Exp decay penalty
    # tradeOff_metric = test_acc * np.exp(-0.5 * min(1, tr_time/theta_parameter))

    # inverse additive penalty
    # tradeOff_metric = test_acc / (1 + 0.5 * min(1, tr_time/theta_parameter))

    # min-based tradeoff
    # tradeOff_metric = min(test_acc, min(1, tr_time/theta_parameter))

    # product with complement
    # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter))

    # square root compensation
    # tradeOff_metric = np.sqrt(test_acc) * np.sqrt(1 - min(1, tr_time/theta_parameter))

    # jacard style normalization
    # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter)) / (test_acc + (1 - min(1, tr_time/theta_parameter)) - test_acc * (1 - min(1, tr_time/theta_parameter)))

    # log-sigmoid dominance
    # tradeOff_metric = 1 / (1 + np.exp(-0.5 * (test_acc - (1 - min(1, tr_time/theta_parameter)))))

    del model

    print("========================== EDW EINAI TO MULTI2222222222.......")
    print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
    print(f"Training time: ", tr_time)
    print(tradeOff_metric)

    q.put([test_acc, tr_time, tradeOff_metric])

  else:
    q.put([0, 1000000000, 0])


In [19]:
def my_evaluate_crosmut(q, x_train, y_train, features_test, labels_test, child_e, child_lr, child_bs, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND, l_child):
  error_flag = -1
  print()
  print(child_e)
  print(child_lr)
  print(child_bs)
  print(x_train.shape)
  print()
  try:
    # We fic the layer2add = 'dense' since we reconstruct the network and we only have to add the last dense for the classification...
    child_model, *_ = recreate_model(l_child, 'dense', CONV_NEURONS_CONST, UNITS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, UNITS_BOUND, DENSE_NEURONS_BOUND)

    # Check if the last layer is a Dense layer
    last_layer = l_child[-1]

    # If the just-added-layer was conv or pool then add manually a flatten layer
    if last_layer == 'conv' or last_layer == 'pool':
      child_model.add(Flatten())

    # Softmax is an activation function that is used mainly for classification tasks
    # It normalizes the input vector into a probability distribution  that is proportional to the exponential of the input numbers.
    child_model.add(tf.keras.layers.Dense(len(CLASSES_LIST), activation = "softmax"))

  except ValueError:
      print("No valid input...:(")
      error_flag = 1

  if error_flag == -1:
    child_model.compile(optimizer=Adam(child_lr), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    child_model.summary()

    # Here we reshape the input of the network based on the type of the first layer of the network
    # If the first layer is conv
    if l_child[0] == 'conv':
      reshaped_x_train = x_train
      reshaped_x_test = features_test
    # If the first layer is lstm-gru-rnn
    elif l_child[0] == 'lstm' or l_child[0] == 'gru' or l_child[0] == 'rnn':
      num_samples, num_frames, height, width, channels = x_train.shape
      reshaped_x_train = x_train.reshape(num_samples, num_frames, height * width * channels)
      num_samples, num_frames, height, width, channels = features_test.shape
      reshaped_x_test = features_test.reshape(num_samples, num_frames, height * width * channels)
    # If the first layer is dense
    else:
      num_samples, num_frames, height, width, channels = x_train.shape
      reshaped_x_train = x_train.reshape(num_samples, num_frames * height * width * channels)
      num_samples, num_frames, height, width, channels = features_test.shape
      reshaped_x_test = features_test.reshape(num_samples, num_frames * height * width * channels)

    start = time.time()

    blackbox = child_model.fit(x=reshaped_x_train,
                        y=y_train,
                        epochs=child_e,
                        batch_size=child_bs
                        )
    stop = time.time()

    # Compute the training speed of this CNN architecture
    tr_time = stop - start

    # Compute the accuracy of our training model in the testing dataset
    test_loss, test_acc = child_model.evaluate(reshaped_x_test,  labels_test, verbose=2)

    # Compute the metric that captures the accuracy--speed tradeoff
    tradeOff_metric = lamda_acc * test_acc - (1 - lamda_acc) * math.tanh(tr_time/theta_parameter - 1)

    # Linear formula
    # tradeOff_metric = lamda_acc * test_acc + (1 - lamda_acc) * (1 - min(1, tr_time/theta_parameter))

    # Multiplicative formula
    # tradeOff_metric = test_acc ** lamda_acc * (1 - min(1, tr_time/theta_parameter)) ** (1 - lamda_acc)

    # Harmonic mean
    # tradeOff_metric = (2 * test_acc * (1 - min(1, tr_time/theta_parameter))) / (test_acc + 1 - min(1, tr_time/theta_parameter) + 1e-5)

    # Sigmoid
    # tradeOff_metric = (1 / (1 + np.exp(-test_acc))) * (1 - 1 / (1 + np.exp(-min(1, tr_time/theta_parameter))))

    # Piecewise
    # tradeOff_metric = test_acc if tr_time <= theta_parameter else test_acc * max(0, 1 - 0.5 * (tr_time - theta_parameter))

    # F1 like
    # alpha = 2
    # tradeOff_metric = ((1 + alpha**2) * test_acc * min(1, tr_time/theta_parameter)) / (alpha ** 2 * test_acc + min(1, tr_time/theta_parameter))

    # Exp decay penalty
    # tradeOff_metric = test_acc * np.exp(-0.5 * min(1, tr_time/theta_parameter))

    # inverse additive penalty
    # tradeOff_metric = test_acc / (1 + 0.5 * min(1, tr_time/theta_parameter))

    # min-based tradeoff
    # tradeOff_metric = min(test_acc, min(1, tr_time/theta_parameter))

    # product with complement
    # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter))

    # square root compensation
    # tradeOff_metric = np.sqrt(test_acc) * np.sqrt(1 - min(1, tr_time/theta_parameter))

    # jacard style normalization
    # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter)) / (test_acc + (1 - min(1, tr_time/theta_parameter)) - test_acc * (1 - min(1, tr_time/theta_parameter)))

    # log-sigmoid dominance
    # tradeOff_metric = 1 / (1 + np.exp(-0.5 * (test_acc - (1 - min(1, tr_time/theta_parameter)))))

    del child_model

    print("========================== EDW EINAI TO MULTI333333.......")
    print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
    print(f"Training time: ", tr_time)
    print(tradeOff_metric)

    q.put([test_acc, tr_time, tradeOff_metric])

  else:
    q.put([0, 1000000000, 0])


In [20]:
def CrossoverMutationResult(df):

  # The first g2g_promotions * g2g_percentage NN architectures will continue to the next generation...
  result = df.head(math.floor(g2g_promotions * g2g_percentage)).copy(deep = True)

  # Create an empty DataFrame with specified column names
  column_names = ['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Accuracy', 'TrainingSpeed', 'Score']
  tmp_df = pd.DataFrame(columns=column_names)

  ##############################################################
  # Crossover and mutation: The first with the remaining g2g_promotions * g2g_percentage architectures, e.g., 1 + 2, 1 + 3, 1 + 4,...
  ##############################################################
  idx = 0
  for i in range(1, (math.floor(g2g_promotions * g2g_percentage))):
    # Get the features of each 'parent', i.e, layers, sampling rates and epochs
    print()
    l1 = df['LayerType'].iloc[idx]
    sr1 = df['SamplingRate'].iloc[idx]
    e1 = df['Epochs'].iloc[idx]
    lr1 = df['LearningRate'].iloc[idx]
    bs1 = df['BatchSize'].iloc[idx]

    l2 = df['LayerType'].iloc[idx + i]
    sr2 = df['SamplingRate'].iloc[idx + i]
    e2 = df['Epochs'].iloc[idx + i]
    lr2 = df['LearningRate'].iloc[idx + i]
    bs2 = df['BatchSize'].iloc[idx + i]

    #  -----------------------------  Crossover  -----------------------------
    # The 'length' of the resulted child will be equal to the min length of its parents
    # For each layer get randomly (50% chance) the corresponding layer of one of its parents
    # Moreover, from one parent get its sampling rate and from the other the number of epochs
    l_min = min(len(l1), len(l2))
    print(l_min)

    l_child = []
    for l in range(0, l_min):

      if l > 0:
        if l_child[-1] == 'dense':
          l_child.append('dense')
          continue
        if l_child[-1] == 'lstm' or l_child[-1] == 'gru' or l_child[-1] == 'rnn':
          select_parent = random.randint(0, 1)
          if select_parent == 0 and (l1[l] != 'conv' and l1[l] != 'pool'):
            l_child.append(l1[l])
          else:
            l_child.append(l2[l])
          continue

      select_parent = random.randint(0, 1)
      if select_parent == 0:
        l_child.append(l1[l])
      else:
        l_child.append(l2[l])

    select_parent = random.randint(0, 5)
    if select_parent == 0:
      child_sr = sr1
      child_e = e1
      child_lr = lr2
      child_bs = bs2
    elif select_parent == 1:
      child_sr = sr2
      child_e = e2
      child_lr = lr1
      child_bs = bs1
    elif select_parent == 2:
      child_sr = sr1
      child_e = e2
      child_lr = lr1
      child_bs = bs2
    elif select_parent == 3:
      child_sr = sr2
      child_e = e1
      child_lr = lr2
      child_bs = bs1
    elif select_parent == 4:
      child_sr = sr1
      child_e = e2
      child_lr = lr2
      child_bs = bs1
    else:
      child_sr = sr2
      child_e = e1
      child_lr = lr1
      child_bs = bs2

    # -----------------------------  Mutation  -----------------------------
    # Perform a mutation in either the sampling rate or the number of epochs.
    select_feature = random.randint(0, 3)
    if select_feature == 0:
      print("Mutation on Sampling Rate")
      random_float = random.uniform(sample_size_low, sample_size_high)
      child_sr = random_float
    elif select_feature == 1:
      print("Mutation on Learning Rate")
      random_float = random.uniform(mutation_lr_low, mutation_lr_high)
      child_lr = random_float
    elif select_feature == 2:
      print("Mutation on batch size")
      random_int = random.randint(mutation_batch_low, mutation_batch_high)
      child_bs = random_int
    else:
      print("Mutation on Epochs")
      random_int = random.randint(num_of_epochs_low, num_of_epochs_high)
      child_e = random_int


    # Perform the sampling
    train_images, train_labels = sampling_method(sampling_method_id, features_train, labels_train, child_sr)
    q = Queue()
    process_eval = multiprocessing.Process(target=my_evaluate_crosmut, args=(q, train_images, train_labels, features_test, labels_test, child_e, child_lr, child_bs, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND, l_child))
    process_eval.start()
    test_acc, tr_time, tradeOff_metric = q.get()
    process_eval.join()

    # Print the results.
    print()
    print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
    print(f"Training time: ", tr_time)
    print(tradeOff_metric)
    print()

    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    gc.collect()
    del train_images
    del train_labels
    #tf.compat.v1.reset_default_graph()

    # Create a dictionary representing a row to add
    new_row_data = {
        'LayerType': l_child,
        'SamplingRate': child_sr,
        'LearningRate': child_lr,
        'BatchSize': child_bs,
        'Epochs': child_e,
        'Accuracy': test_acc,
        'TrainingSpeed': tr_time,
        'Score': tradeOff_metric
    }

    # Create a new DataFrame with the single row
    new_row_df = pd.DataFrame([new_row_data])

    # Concatenate the empty DataFrame and the new DataFrame
    tmp_df = pd.concat([tmp_df, new_row_df], ignore_index=True)

    print("-----------------> End of first phase crossovers/mutations")

  print(tmp_df)

  ##############################################################
  # Crossover and mutation topK with downK, e.g., 1 + 6, 2 + 7, 3 = 8, 4 + 9 and 5 + 10
  ##############################################################

  print(df)
  for i in range(math.floor(len(df) * 0.5)):
    print()
    l1 = df['LayerType'].iloc[i]
    sr1 = df['SamplingRate'].iloc[i]
    e1 = df['Epochs'].iloc[i]
    lr1 = df['LearningRate'].iloc[i]
    bs1 = df['BatchSize'].iloc[i]

    l2 = df['LayerType'].iloc[i + math.floor(len(df) * 0.5)]
    sr2 = df['SamplingRate'].iloc[i + math.floor(len(df) * 0.5)]
    e2 = df['Epochs'].iloc[i + math.floor(len(df) * 0.5)]
    lr2 = df['LearningRate'].iloc[i + math.floor(len(df) * 0.5)]
    bs2 = df['BatchSize'].iloc[i + math.floor(len(df) * 0.5)]

    # Crossover (similar to the first phase crossover)
    l_min = min(len(l1), len(l2))
    print(l_min)

    l_child = []
    for l in range(0, l_min):

      if l > 0:
        if l_child[-1] == 'dense':
          l_child.append('dense')
          continue
        if l_child[-1] == 'lstm' or l_child[-1] == 'gru' or l_child[-1] == 'rnn':
            select_parent = random.randint(0, 1)
            if select_parent == 0 and (l1[l] != 'conv' and l1[l] != 'pool'):
              l_child.append(l1[l])
            else:
              l_child.append(l2[l])
            continue

      select_parent = random.randint(0, 1)
      if select_parent == 0:
        l_child.append(l1[l])
      else:
        l_child.append(l2[l])

    select_parent = random.randint(0, 5)
    if select_parent == 0:
      child_sr = sr1
      child_e = e1
      child_lr = lr2
      child_bs = bs2
    elif select_parent == 1:
      child_sr = sr2
      child_e = e2
      child_lr = lr1
      child_bs = bs1
    elif select_parent == 2:
      child_sr = sr1
      child_e = e2
      child_lr = lr1
      child_bs = bs2
    elif select_parent == 3:
      child_sr = sr2
      child_e = e1
      child_lr = lr2
      child_bs = bs1
    elif select_parent == 4:
      child_sr = sr1
      child_e = e2
      child_lr = lr2
      child_bs = bs1
    else:
      child_sr = sr2
      child_e = e1
      child_lr = lr1
      child_bs = bs2

    # -----------------------------  Mutation  -----------------------------
    # Perform a mutation in either the sampling rate or the number of epochs.
    select_feature = random.randint(0, 3)
    if select_feature == 0:
      print("Mutation on Sampling Rate")
      random_float = random.uniform(sample_size_low, sample_size_high)
      child_sr = random_float
    elif select_feature == 1:
      print("Mutation on Learning Rate")
      random_float = random.uniform(mutation_lr_low, mutation_lr_high)
      child_lr = random_float
    elif select_feature == 2:
      print("Mutation on batch size")
      random_int = random.randint(mutation_batch_low, mutation_batch_high)
      child_bs = random_int
    else:
      print("Mutation on Epochs")
      random_int = random.randint(num_of_epochs_low, num_of_epochs_high)
      child_e = random_int


    # Perform the sampling
    train_images, train_labels = sampling_method(sampling_method_id, features_train, labels_train, child_sr)
    q = Queue()
    process_eval = multiprocessing.Process(target=my_evaluate_crosmut, args=(q, train_images, train_labels, features_test, labels_test, child_e, child_lr, child_bs, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND, l_child))
    process_eval.start()
    test_acc, tr_time, tradeOff_metric = q.get()
    process_eval.join()

    # Print the results.
    print()
    print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
    print(f"Training time: ", tr_time)
    print(tradeOff_metric)
    print()

    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    gc.collect()
    del train_images
    del train_labels
    #tf.compat.v1.reset_default_graph()

    # Create a dictionary representing a row to add
    new_row_data = {
        'LayerType': l_child,
        'SamplingRate': child_sr,
        'LearningRate': child_lr,
        'BatchSize': child_bs,
        'Epochs': child_e,
        'Accuracy': test_acc,
        'TrainingSpeed': tr_time,
        'Score': tradeOff_metric
    }

    # Create a new DataFrame with the single row
    new_row_df = pd.DataFrame([new_row_data])

    # Concatenate the empty DataFrame and the new DataFrame
    tmp_df = pd.concat([tmp_df, new_row_df], ignore_index=True)

    print("-----------------> End of second phase crossovers/mutations")

  print(tmp_df)

  ###################################################
  # Add the original networks (positions that are after the first g2g_percentage), e.g. if g2g_rercentage is 0.2 and g2g_promotion = 10, then you will add the architectures in positions 3, 4, ..., 10
  ###################################################

  tmp_df = pd.concat([tmp_df, df_first.iloc[math.floor(g2g_promotions * g2g_percentage):]])
  print("-----------------> Add the original networks (positions after the first g2g_percentage)")
  print(tmp_df)

  tmp_df = tmp_df.drop(columns='CD')
  tmp_df = tmp_df.reset_index(drop=True)

  print(tmp_df)

  ###################################################
  # Compute Crowding Distance
  ###################################################

  objective_columns = ['Accuracy', 'TrainingSpeed']
  crowding_distances = CrowdingDist(tmp_df, objective_columns)

  tmp_df['CD'] = crowding_distances
  tmp_df.sort_values('CD', ascending = False, inplace = True)

  tmp_df.reset_index(drop = True, inplace = True)

  # Get the (1 - g2g_percentage) architectures with the highest Crowding Distance
  df_cd_crosmut = tmp_df.head(math.ceil(g2g_promotions * (1 - g2g_percentage)))


  ###################################################
  # Create the final result
  ###################################################
  result = pd.concat([result, df_cd_crosmut])
  result = result.drop(columns='CD')
  result.reset_index(drop = True, inplace = True)

  return result

In [21]:
start_program = time.time()

# Get the shape of the input dataset
dataset_shape = features_train.shape

# Store the best type of layer of each step
best_layers = []

# Loop for the number of layers that user indicated
for step in range(1, total_num_of_layers + 1):

  # If this is the first step/iteration
  if step == 1:

    # The first layer in this set of datasets has to be a convolutional one!
    # So our "search space" all combinations of epochs and sampling rates based on user's input
    for sample_size in np.arange(sample_size_low, sample_size_high + 0.01, sample_size_step):
      for epochs in range(num_of_epochs_low, num_of_epochs_high + 1, num_of_epochs_step):
        for lr in lr_list:
          for size_of_batch in range(size_of_batch_low, size_of_batch_high + 1, size_of_batch_step):
            for layer in set_of_layers:

              # The first layer can't be pooling so skip this iteration
              if layer != 'conv':
                continue

              # Perform the sampling
              x_train, y_train = sampling_method(sampling_method_id, features_train, labels_train, sample_size)

              q = Queue()
              process_eval = multiprocessing.Process(target=my_evaluate_first_phase, args=(q, layer, x_train, y_train, features_test, labels_test, epochs, lr, size_of_batch))
              process_eval.start()
              test_acc, tr_time, tradeOff_metric = q.get()
              process_eval.join()

              # Print the results.
              print()
              print("========================== EDW EINAI TO CORE.......")
              print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
              print(f"Training time: ", tr_time)
              print(tradeOff_metric)
              print()

              # Delete the dfs.
              del x_train
              del y_train

              # Clear the Keras session, otherwise it will keep adding new
              # models to the same TensorFlow graph each time we create
              # a model with a different set of hyper-parameters.
              K.clear_session()
              tf.compat.v1.reset_default_graph()

              # If this is the first combination that is examined then create the dataframe and store the result
              if sample_size == sample_size_low and epochs == num_of_epochs_low and lr == lr_list[0] and size_of_batch == size_of_batch_low and layer == set_of_layers[0]:
                df = pd.DataFrame([[[layer], sample_size, lr, size_of_batch, epochs, test_acc, tr_time, tradeOff_metric]], columns=['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Accuracy', 'TrainingSpeed', 'Score'])
              # else append the result to the existing dataframe
              else:
                df_tmp = pd.DataFrame([[[layer], sample_size, lr, size_of_batch, epochs, test_acc, tr_time, tradeOff_metric]], columns=['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Accuracy', 'TrainingSpeed', 'Score'])
                df = pd.concat([df, df_tmp], axis = 0, ignore_index=True)
                del df_tmp

  # If this is not the first step/iteration
  else:
    # This boolean variable indicates if this is the first examined combination in order to create a df
    flag = True

    # Get the df of the previous step/iteration
    df_tmp2 = best_layers[-1]

    # For every pareto optimal solution (of the previous step) except the top% that remains the same
    for pareto_layer in range(math.floor(g2g_promotions * g2g_percentage), len(df_tmp2)):
      current_df = df_tmp2.iloc[pareto_layer]

      # For all the available types of layers
      for layer in set_of_layers:

        # After a dense laye you can only add a dense layer
        # So, don't examine the other types of layers
        if current_df['LayerType'][-1] == 'dense' and layer != 'dense':
          continue

        # If the last added layer is lstm-gru-rnn you cannot add as next layer both conv and pool layers so skip these iterations
        if ((current_df['LayerType'][-1] == 'lstm' or current_df['LayerType'][-1] == 'gru' or current_df['LayerType'][-1] == 'rnn') and (layer == 'conv' or layer == 'pool')):
          continue

        # Perform the sampling
        x_train, y_train = sampling_method(sampling_method_id, features_train, labels_train, current_df['SamplingRate'])

        q = Queue()
        process_eval = multiprocessing.Process(target=my_evaluate_rest_phase, args=(q, x_train, y_train, features_test, labels_test, layer, current_df, CONV_NEURONS_CONST, UNITS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, UNITS_BOUND, DENSE_NEURONS_BOUND))
        process_eval.start()
        test_acc, tr_time, tradeOff_metric = q.get()
        process_eval.join()

        # Print the results.
        print()
        print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
        print(f"Training time: ", tr_time)
        print(tradeOff_metric)
        print()

        # Delete the Keras model with these hyper-parameters from memory.
        K.clear_session()
        gc.collect()
        del x_train
        del y_train

        # Clear the Keras session, otherwise it will keep adding new
        # models to the same TensorFlow graph each time we create
        # a model with a different set of hyper-parameters.
        tf.compat.v1.reset_default_graph()

        # If this is the first combination that is examined then create the dataframe and store the result
        if flag:
          flag = False
          so_far_layers = current_df['LayerType'].copy()
          so_far_layers.append(layer)
          df = pd.DataFrame([[so_far_layers, current_df['SamplingRate'], current_df['LearningRate'], current_df['BatchSize'], current_df['Epochs'], test_acc, tr_time, tradeOff_metric]], columns=['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Accuracy', 'TrainingSpeed', 'Score'])
        # else append the result to the existing df
        else:
          so_far_layers = current_df['LayerType'].copy()
          so_far_layers.append(layer)
          df_tmp = pd.DataFrame([[so_far_layers, current_df['SamplingRate'], current_df['LearningRate'], current_df['BatchSize'], current_df['Epochs'], test_acc, tr_time, tradeOff_metric]], columns=['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Accuracy', 'TrainingSpeed', 'Score'])
          df = pd.concat([df, df_tmp], axis = 0, ignore_index=True)
          del df_tmp

  # If this is after the first iteration-layer, then the best g2g_promotions * g2g_percentage are not expanded and as such we append them directly to the resulted df
  # Note that df_tmp2 is the dataframe that contains the best solutions from the previous iteration-layer
  if step > 1:
    print(df_tmp2)
    df = pd.concat([df_tmp2.iloc[:math.floor(g2g_promotions * g2g_percentage)], df], ignore_index=True)
    print(df)

  # flag that indicates if the to-be-computed rank solutions (pareto front) is the first one
  first_rank = True
  counter = 0

  while True:

    counter += 1

    print("######################")
    print("Our candidates")
    print("######################")
    print(df)

    # Keep the columns of the dataframe that we need to compute the Pareto opt, i.e., accuracy and training speed
    df2 = df.loc[:, df.columns.drop(['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Score'])]

    # Set the appropriate mask, i.e., max the accuracy, minimize the training speed
    mask = paretoset(df2, sense=["max", "min"])

    # Apply the mask and get the result
    paretoset_nn = df[mask]
    paretoset_nn.reset_index(drop = True, inplace = True)

    # If this is the first rank create the df, else just append the result to the existing df
    if first_rank:
      my_ranks = copy.deepcopy(paretoset_nn)
      first_rank = False
    else:
      my_ranks = pd.concat([my_ranks, paretoset_nn], ignore_index=True)

    print("######################")
    print(f"The {counter}-ranks that have qualified (previous ranks are also included if they exist...)")
    print("######################")
    print(my_ranks)

    # Check if you need the next rank
    if my_ranks.shape[0] >= g2g_promotions:
      print("----------------------> We do not need the next rank...")
      break

    else:
      #####################################################################
      #     Remove the ith rank in order to compute the ith + 1 rank
      #####################################################################
      df['LayerType'] = df['LayerType'].apply(tuple)
      paretoset_nn['LayerType'] = paretoset_nn['LayerType'].apply(tuple)

      result_df = pd.merge(df, paretoset_nn, on=['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Accuracy', 'TrainingSpeed', 'Score'], how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

      result_df['LayerType'] = result_df['LayerType'].apply(list)
      df['LayerType'] = df['LayerType'].apply(list)
      paretoset_nn['LayerType'] = paretoset_nn['LayerType'].apply(list)

      df = copy.deepcopy(result_df)

  # Compute the crowding distance between all the architectures that were qualified based on their rank
  objective_columns = ['Accuracy', 'TrainingSpeed']
  crowding_distances = CrowdingDist(my_ranks, objective_columns)

  my_ranks['CD'] = crowding_distances
  my_ranks.sort_values('CD', ascending = False, inplace = True)
  print(my_ranks)

  # Selecting the top g2g_promotions * g2g_percentage rows with the maximum 'f1'
  top_max_score = my_ranks.nlargest(math.floor(g2g_promotions * g2g_percentage), 'Score')

  my_ranks['LayerType'] = my_ranks['LayerType'].apply(tuple)
  top_max_score['LayerType'] = top_max_score['LayerType'].apply(tuple)

  # Adding the top g2g_promotions * g2g_percentage rows to the top of the original DataFrame and removing duplicates
  my_ranks = pd.concat([top_max_score, my_ranks]).drop_duplicates().reset_index(drop=True)

  my_ranks['LayerType'] = my_ranks['LayerType'].apply(list)
  top_max_score['LayerType'] = top_max_score['LayerType'].apply(list)

  my_ranks.reset_index(drop = True, inplace = True)
  print(my_ranks)

  # Qualify the best based on ther CD score
  df_first = my_ranks.head(g2g_promotions)
  print(df_first)

  # Crossover + Mutation function
  df_crossmut = CrossoverMutationResult(df_first)

  # Add the dataframe that contains the pareto opt architectures for this step/iteration and the corresponding metrics
  best_layers.append(df_crossmut)


Percentage of filtering in our training dataset was set:
0.05
Pre-sampling training data:
Class 0 has 124 samples in our dataset...
Class 1 has 76 samples in our dataset...
Class 2 has 82 samples in our dataset...
Class 3 has 104 samples in our dataset...
Class 4 has 113 samples in our dataset...
Class 5 has 78 samples in our dataset...
Class 6 has 120 samples in our dataset...
Class 7 has 75 samples in our dataset...
Class 8 has 107 samples in our dataset...
Class 9 has 105 samples in our dataset...
Post-sampling dataset:
Class 0 has 6 samples in our dataset...
Class 1 has 3 samples in our dataset...
Class 2 has 4 samples in our dataset...
Class 3 has 5 samples in our dataset...
Class 4 has 5 samples in our dataset...
Class 5 has 3 samples in our dataset...
Class 6 has 6 samples in our dataset...
Class 7 has 3 samples in our dataset...
Class 8 has 5 samples in our dataset...
Class 9 has 5 samples in our dataset...
Training dataset before sampling:
(984, 20, 64, 64, 3)
(984,)
Training 

  super().__init__(**kwargs)



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 20, 64, 64, 16) │           448 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1310720)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │    13,107,210 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,107,658 (50.00 MB)

...

 Trainable params: 41,968,042 (160.10 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/14


  output, from_logits = _get_logits(



46/46 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.1606 - loss: 6.9033
Epoch 2/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6121 - loss: 1.2425
Epoch 3/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8396 - loss: 0.5087
Epoch 4/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8963 - loss: 0.3031
Epoch 5/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9290 - loss: 0.2187
Epoch 6/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9836 - loss: 0.1148
Epoch 7/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 0.0106
Epoch 8/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 0.0077
Epoch 9/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9716 - loss: 0.1201
Epoch 10/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9788 - loss: 0.1879
Epoch 11/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9937 - loss: 0.0224
Epoch 12/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 1.0000 - 

In [22]:
stop_program = time.time()

In [23]:
for i in range(1, len(best_layers)):
  if i == 1:
    final_results = pd.concat([best_layers[0], best_layers[i]], axis=0)
  else:
    final_results = pd.concat([final_results, best_layers[i]], axis=0)

In [24]:
final_results

,LayerType,SamplingRate,LearningRate,BatchSize,Epochs,Accuracy,TrainingSpeed,Score
0,[conv],0.350000,0.004414,10,13,0.423781,8.987221,0.133218
1,[conv],0.350000,0.001000,10,5,0.545732,6.795787,0.333472
2,[conv],0.950000,0.001000,10,9,0.716463,12.081853,-0.113060
3,[conv],0.650000,0.001000,10,5,0.698171,10.297652,0.043036
4,[conv],0.350000,0.001000,10,9,0.603659,7.908589,0.245896
5,[conv],0.650000,0.001000,10,4,0.640244,9.867678,0.075933
0,"[conv, conv]",0.350000,0.001000,45,5,0.490854,14.705772,-0.345714
1,"[conv, conv]",0.350000,0.001000,10,5,0.628049,10.158780,0.048516
2,"[conv, pool]",0.350000,0.001000,10,9,0.713415,8.929706,0.167302
3,"[conv, pool]",0.950000,0.001000,10,9,0.814024,12.715838,-0.157186


In [25]:
df_best = best_layers[-1].query('Score == Score.max()')
df_best

,LayerType,SamplingRate,LearningRate,BatchSize,Epochs,Accuracy,TrainingSpeed,Score
4,"[conv, pool, conv, pool, dense]",0.35,0.001,10,14,0.707317,15.011838,-0.346011


In [26]:
evolution_solution = df_best['LayerType'].tolist()
evolution_solution[0]

['conv', 'pool', 'conv', 'pool', 'dense']

In [27]:
if evolution_solution[0][-1] == 'conv' or evolution_solution[0][-1] == 'pool':
  evolution_solution[0].append('flatten')
evolution_solution[0].append('dense_classes_softmax')

In [28]:
print(f"Based on the evolutionary approach the best nework is ------> {evolution_solution[0]}")

Based on the evolutionary approach the best nework is ------> ['conv', 'pool', 'conv', 'pool', 'dense', 'dense_classes_softmax']


In [29]:
# Compute the time
program_time = stop_program - start_program
program_time

3912.727399110794